In [162]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import geopy.distance
from geopy.extra.rate_limiter import RateLimiter
import json
import glob
import os
from difflib import SequenceMatcher
geolocator=Nominatim(user_agent="bike_application")
trips11=pd.read_csv(r"C:\Users\giorg\Desktop\Trips\capitalbikeshare_tripdata_11.csv")
files=os.path.join("C:\\Users\\giorg\\Desktop\\Trips\\","capitalbikeshare_tripdata_12q*.csv")
files=glob.glob(files)
trips12=pd.concat(map(pd.read_csv,files),ignore_index=True)
trips11.columns=trips11.columns.str.replace(" ","_").str.lower()
trips12.columns=trips12.columns.str.replace(" ","_").str.lower()
trips11["start_date"]=pd.to_datetime(trips11["start_date"])
trips11["end_date"]=pd.to_datetime(trips11["end_date"])
trips11["duration"]=(trips11["duration"]/60).round(1)
trips12["start_date"]=pd.to_datetime(trips12["start_date"])
trips12["end_date"]=pd.to_datetime(trips12["end_date"])
trips12["duration"]=(trips12["duration"]/60).round(1)
stations11=trips11["start_station"].unique()
stations11=pd.DataFrame(stations11,columns=["station"])
stations11["lat"]=np.nan
stations11["long"]=np.nan
stations12=trips12["start_station"].unique()
stations12=pd.DataFrame(stations12,columns=["station"])
stations12["lat"]=np.nan
stations12["long"]=np.nan

In [163]:
trips11_new=pd.read_csv(r"C:\Users\giorg\Desktop\trips11_new")

In [164]:
def dist_cald(indexer1):
    coords1=(trips11_new.loc[indexer1,"lat_start"],trips11_new.loc[indexer1,"long_start"])
    coords2=(trips11_new.loc[indexer1,"lat_end"],trips11_new.loc[indexer1,"long_end"])
    return geopy.distance.distance(coords1,coords2)

In [165]:
with open("en_station_information.json","r") as json_file:
    json_load=json.load(json_file)
station_info=pd.DataFrame(json_load["data"]["stations"])
station_info=station_info.drop(["electric_bike_surcharge_waiver","eightd_station_services","station_type","rental_uris","rental_methods",
"external_id","has_kiosk","eightd_has_key_dispenser"],axis=1)

In [166]:
station_info["region_id"]=station_info["region_id"].fillna(0)
station_info["region_id"]=station_info["region_id"].astype("int64")
station_info["region"]=station_info["region_id"].map({40:"Alexandria Virginia",41:"Arlington Virginia",42:"Washington DC",43:"Montgomery County Maryland",
44:"Montgomery Country Maryland",48:"Test and Operations",104:"Fairfax Virginia",128:"8D",133:"Prince George\'s County Maryland",152:"Falls Church Virginia"}).astype("category")

In [167]:
for i in range(len(stations11)):
    for j in range(len(station_info)):
        if stations11.loc[i,"station"]==station_info.loc[j,"name"]:
            stations11.loc[i,"lat"]=station_info.loc[j,"lat"]
            stations11.loc[i,"long"]=station_info.loc[j,"lon"]

stations11=stations11.fillna(0)
stations11["region"]=0
#We see that there are slight differences in the adresses spelling, so we consider adresses with more than 80% similarity to be the same
for i in range(len(stations11)):
    for j in range(len(station_info)):
        if (SequenceMatcher(None, stations11.loc[i,"station"],station_info.loc[j,"name"])).ratio()>0.8:
            stations11.loc[i,"region"]=" "
            stations11.loc[i,"region"]=stations11.loc[i,"region"]+station_info.loc[j,"region"]
#Now we find the coordinates for those adresses
geocode=RateLimiter(geolocator.geocode, min_delay_seconds=1)
for i in range(len(stations11)):
    if stations11.loc[i,"lat"]==0 or stations11.loc[i,"long"]==0:
        if stations11.loc[i,"region"]==0:
            a=geocode(stations11.loc[i,"station"],timeout=None)    
        else:
            a=geocode((stations11.loc[i,"station"]+stations11.loc[i,"region"]),timeout=None)
        try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations11.loc[i,"lat"]=la
                stations11.loc[i,"long"]=lo
        except AttributeError:
            pass
for i in range(len(stations11)):
    if stations11.loc[i,"lat"]==0 or stations11.loc[i,"long"]==0:
        if "/" in stations11.loc[i,"station"]:
            j,x=((stations11.loc[i,"station"]).split("/"))
            a=geocode(x)
        else:
            a=geocode(stations11.loc[i,"station"])
    try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations11.loc[i,"lat"]=la
                stations11.loc[i,"long"]=lo
    except AttributeError:
            pass
for i in range(len(stations11)):
    if stations11.loc[i,"lat"]==0 or stations11.loc[i,"long"]==0:
        a=geocode(stations11.loc[i,"station"])
        try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations11.loc[i,"lat"]=la
                stations11.loc[i,"long"]=lo
        except AttributeError:
            pass

In [169]:
for i in range(len(stations12)):
    for j in range(len(station_info)):
        if stations12.loc[i,"station"]==station_info.loc[j,"name"]:
            stations12.loc[i,"lat"]=station_info.loc[j,"lat"]
            stations12.loc[i,"long"]=station_info.loc[j,"lon"]

stations12=stations12.fillna(0)
stations12["region"]=0
#We see that there are slight differences in the adresses spelling, so we consider adresses with more than 80% similarity to be the same
for i in range(len(stations12)):
    for j in range(len(station_info)):
        if (SequenceMatcher(None, stations12.loc[i,"station"],station_info.loc[j,"name"])).ratio()>0.8:
            stations12.loc[i,"region"]=" "
            stations12.loc[i,"region"]=stations12.loc[i,"region"]+station_info.loc[j,"region"]
#Now we find the coordinates for those adresses
geocode=RateLimiter(geolocator.geocode, min_delay_seconds=1)
for i in range(len(stations12)):
    if stations12.loc[i,"lat"]==0 or stations12.loc[i,"long"]==0:
        if stations12.loc[i,"region"]==0:
            a=geocode(stations12.loc[i,"station"],timeout=None)    
        else:
            a=geocode((stations12.loc[i,"station"]+stations12.loc[i,"region"]),timeout=None)
        try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations12.loc[i,"lat"]=la
                stations12.loc[i,"long"]=lo
        except AttributeError:
            pass
for i in range(len(stations12)):
    if stations12.loc[i,"lat"]==0 or stations12.loc[i,"long"]==0:
        if "/" in stations12.loc[i,"station"]:
            j,x=((stations12.loc[i,"station"]).split("/"))
            a=geocode(x)
        else:
            a=geocode(stations12.loc[i,"station"])
    try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations12.loc[i,"lat"]=la
                stations12.loc[i,"long"]=lo
    except AttributeError:
            pass
for i in range(len(stations12)):
    if stations12.loc[i,"lat"]==0 or stations12.loc[i,"long"]==0:
        a=geocode(stations12.loc[i,"station"])
        try:
            la=a.latitude
            lo=a.longitude
            if (la>38 and la<39.1) and (lo>-77.3 and lo<-76.6):
                stations12.loc[i,"lat"]=la
                stations12.loc[i,"long"]=lo
        except AttributeError:
            pass

In [ ]:
#Takes centuries to run, use it only during night (122΄  to complete)
# trips11["lat_start"]=0
# trips11["long_start"]=0
# trips11["lat_end"]=0
# trips11["long_end"]=0
# for i in range(len(trips11)):
#     for j in range(len(stations11)):
#         if trips11.loc[i,"start_station"]==stations11.loc[j,"station"]:
#             trips11.loc[i,"lat_start"]=stations11.loc[j,"lat"]
#             trips11.loc[i,"long_start"]=stations11.loc[j,"long"]
#         if trips11.loc[i,"end_station"]==stations11.loc[j,"station"]:
#             trips11.loc[i,"lat_end"]=stations11.loc[j,"lat"]
#             trips11.loc[i,"long_end"]=stations11.loc[j,"long"]
            

In [ ]:
#Bounding box coordinates
# i=stations11.loc[stations11["lat"]!=0]
# j=stations11.loc[stations11["long"]!=0]
# bbox=(i["lat"].max(),i["lat"].min(),j["long"].max(),j["long"].min())

In [190]:
temp=trips11.loc[(trips11["duration"]<3)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp=temp1.loc[(temp1["start_station"]==temp1["end_station"])&(trips11["duration"]<5)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp1.loc[(temp1["member_type"]=="Member")]
temp1["month"]=pd.DatetimeIndex(temp1["start_date"]).month
temp1=temp1.groupby(["start_station","month"]).size()
temp1=pd.DataFrame(temp1)
temp1.reset_index(inplace=True)
for i in range(len(stations11)):
    for j in range(len(temp1)):
        if stations11.loc[i,"station"] == temp1.loc[j,"start_station"]:
            temp1.loc[j,"lat"]=stations11.loc[i,"lat"]
            temp1.loc[j,"long"]=stations11.loc[i,"long"]
temp=trips12.loc[(trips12["duration"]<3)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp=temp2.loc[(temp2["start_station"]==temp2["end_station"])&(trips12["duration"]<5)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp2.loc[(temp2["member_type"]=="Member")]
temp2["month"]=pd.DatetimeIndex(temp2["start_date"]).month
temp2=temp2.groupby(["start_station","month"]).size()
temp2=pd.DataFrame(temp2)
temp2.reset_index(inplace=True)
for i in range(len(stations12)):
    for j in range(len(temp2)):
        if stations12.loc[i,"station"] == temp2.loc[j,"start_station"]:
            temp2.loc[j,"lat"]=stations12.loc[i,"lat"]
            temp2.loc[j,"long"]=stations12.loc[i,"long"]
temp2["month"]+=12
temp1=pd.concat([temp1,temp2])
temp1.to_csv("member_usage_start")

temp=trips11.loc[(trips11["duration"]<3)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp=temp1.loc[(temp1["start_station"]==temp1["end_station"])&(trips11["duration"]<5)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp1.loc[(temp1["member_type"]=="Casual")]
temp1["month"]=pd.DatetimeIndex(temp1["start_date"]).month
temp1=temp1.groupby(["start_station","month"]).size()
temp1=pd.DataFrame(temp1)
temp1.reset_index(inplace=True)
for i in range(len(stations11)):
    for j in range(len(temp1)):
        if stations11.loc[i,"station"] == temp1.loc[j,"start_station"]:
            temp1.loc[j,"lat"]=stations11.loc[i,"lat"]
            temp1.loc[j,"long"]=stations11.loc[i,"long"]
temp=trips12.loc[(trips12["duration"]<3)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp=temp2.loc[(temp2["start_station"]==temp2["end_station"])&(trips12["duration"]<5)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp2.loc[(temp2["member_type"]=="Casual")]
temp2["month"]=pd.DatetimeIndex(temp2["start_date"]).month
temp2=temp2.groupby(["start_station","month"]).size()
temp2=pd.DataFrame(temp2)
temp2.reset_index(inplace=True)
for i in range(len(stations12)):
    for j in range(len(temp2)):
        if stations12.loc[i,"station"] == temp2.loc[j,"start_station"]:
            temp2.loc[j,"lat"]=stations12.loc[i,"lat"]
            temp2.loc[j,"long"]=stations12.loc[i,"long"]
temp2["month"]+=12
temp1=pd.concat([temp1,temp2])
temp1.to_csv("casual_usage_start")


In [191]:
temp=trips11.loc[(trips11["duration"]<3)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp=temp1.loc[(temp1["start_station"]==temp1["end_station"])&(trips11["duration"]<5)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp1.loc[(temp1["member_type"]=="Member")]
temp1["month"]=pd.DatetimeIndex(temp1["start_date"]).month
temp1=temp1.groupby(["end_station","month"]).size()
temp1=pd.DataFrame(temp1)
temp1.reset_index(inplace=True)
for i in range(len(stations11)):
    for j in range(len(temp1)):
        if stations11.loc[i,"station"] == temp1.loc[j,"end_station"]:
            temp1.loc[j,"lat"]=stations11.loc[i,"lat"]
            temp1.loc[j,"long"]=stations11.loc[i,"long"]
temp=trips12.loc[(trips12["duration"]<3)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp=temp2.loc[(temp2["start_station"]==temp2["end_station"])&(trips12["duration"]<5)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp2.loc[(temp2["member_type"]=="Member")]
temp2["month"]=pd.DatetimeIndex(temp2["start_date"]).month
temp2=temp2.groupby(["end_station","month"]).size()
temp2=pd.DataFrame(temp2)
temp2.reset_index(inplace=True)
for i in range(len(stations12)):
    for j in range(len(temp2)):
        if stations12.loc[i,"station"] == temp2.loc[j,"end_station"]:
            temp2.loc[j,"lat"]=stations12.loc[i,"lat"]
            temp2.loc[j,"long"]=stations12.loc[i,"long"]
temp2["month"]+=12
temp1=pd.concat([temp1,temp2])
temp1.to_csv("member_usage_end")

temp=trips11.loc[(trips11["duration"]<3)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp=temp1.loc[(temp1["start_station"]==temp1["end_station"])&(trips11["duration"]<5)]
temp1=pd.concat([trips11,temp,temp]).drop_duplicates(keep=False)
temp1.loc[(temp1["member_type"]=="Casual")]
temp1["month"]=pd.DatetimeIndex(temp1["start_date"]).month
temp1=temp1.groupby(["end_station","month"]).size()
temp1=pd.DataFrame(temp1)
temp1.reset_index(inplace=True)
for i in range(len(stations11)):
    for j in range(len(temp1)):
        if stations11.loc[i,"station"] == temp1.loc[j,"end_station"]:
            temp1.loc[j,"lat"]=stations11.loc[i,"lat"]
            temp1.loc[j,"long"]=stations11.loc[i,"long"]
temp=trips12.loc[(trips12["duration"]<3)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp=temp2.loc[(temp2["start_station"]==temp2["end_station"])&(trips12["duration"]<5)]
temp2=pd.concat([trips12,temp,temp]).drop_duplicates(keep=False)
temp2.loc[(temp2["member_type"]=="Casual")]
temp2["month"]=pd.DatetimeIndex(temp2["start_date"]).month
temp2=temp2.groupby(["end_station","month"]).size()
temp2=pd.DataFrame(temp2)
temp2.reset_index(inplace=True)
for i in range(len(stations12)):
    for j in range(len(temp2)):
        if stations12.loc[i,"station"] == temp2.loc[j,"end_station"]:
            temp2.loc[j,"lat"]=stations12.loc[i,"lat"]
            temp2.loc[j,"long"]=stations12.loc[i,"long"]
temp2["month"]+=12
temp1=pd.concat([temp1,temp2])
temp1.to_csv("casual_usage_end")